In [1]:
import pandas as pd
from nlpia.data.loaders import get_data
pd.options.display.width=120
sms=get_data('sms-spam')
index=['sms{}{}'.format(i,'!'*j) for (i,j) in zip(range(len(sms)),sms.spam)]
sms.index=index
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf=TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs=tfidf.fit_transform(raw_documents=sms.text).toarray()
tfidf_docs=tfidf_docs-tfidf_docs.mean(axis=0)

In [2]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(tfidf_docs,sms.spam.values,test_size=0.2,random_state=271828)


In [3]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda=LDA(n_components=1)

In [4]:
lda.fit(x_train,y_train)

LinearDiscriminantAnalysis(n_components=1)

In [5]:
round(float(lda.score(x_train,y_train)),3)


1.0

In [7]:
round(float(lda.score(x_test,y_test)),3)

0.764

In [13]:
tfidf_docs1=tfidf.fit_transform(raw_documents=sms.text).toarray()


In [ ]:
tfidf_docs1

In [14]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia=LDiA(n_components=32,learning_method='batch')
ldia=ldia.fit(tfidf_docs1)
ldia.components_.shape

(32, 9232)

In [16]:
ldia_topic_vectors=ldia.transform(tfidf_docs1)
columns32=['topic{}'.format(i) for i in range(ldia.n_components)]
ldia_topic_vectors=pd.DataFrame(ldia_topic_vectors,index=index,columns=columns32)

In [17]:
ldia_topic_vectors.round(3).head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic22,topic23,topic24,topic25,topic26,topic27,topic28,topic29,topic30,topic31
sms0,0.006,0.112,0.006,0.006,0.006,0.006,0.006,0.006,0.006,0.006,...,0.006,0.006,0.329,0.006,0.006,0.006,0.006,0.006,0.006,0.006
sms1,0.009,0.009,0.009,0.009,0.009,0.009,0.009,0.009,0.009,0.009,...,0.009,0.009,0.009,0.009,0.009,0.009,0.009,0.009,0.009,0.284
sms2!,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,...,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.005
sms3,0.008,0.759,0.008,0.008,0.008,0.008,0.008,0.008,0.008,0.008,...,0.008,0.008,0.008,0.008,0.008,0.008,0.008,0.008,0.008,0.008
sms4,0.007,0.007,0.174,0.007,0.007,0.007,0.007,0.007,0.007,0.007,...,0.007,0.007,0.007,0.007,0.007,0.007,0.007,0.007,0.007,0.007


In [18]:
x_train,x_test,y_train,y_test=train_test_split(ldia_topic_vectors,sms.spam.values,test_size=0.2,random_state=271828)

In [20]:
lda=LDA(n_components=1)
lda=lda.fit(x_train,y_train)
sms['ldia32_spam']=lda.predict(ldia_topic_vectors)
x_train.shape

(3869, 32)

In [21]:
round(float(lda.score(x_train,y_train)),3)

0.88

In [22]:
round(float(lda.score(x_test,y_test)),3)

0.887

In [33]:
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=32,n_iter=100)
svd_topic_vectors=svd.fit_transform(tfidf_docs)

In [34]:
svd_topic_vectors

array([[ 2.01169147e-01,  2.77055415e-03,  3.72344467e-02, ...,
        -1.82370549e-02, -1.32671462e-02,  7.31354557e-03],
       [ 4.04379632e-01, -9.38752878e-02, -7.75125402e-02, ...,
        -3.35126700e-02, -3.50042061e-02, -1.16406022e-02],
       [-3.04586449e-02, -4.81044287e-02,  9.01907210e-02, ...,
         1.73558412e-02, -3.43129436e-02,  7.23867110e-03],
       ...,
       [ 7.67081459e-02,  4.33699856e-02, -1.93974884e-02, ...,
         7.00217900e-03, -6.32667291e-02,  1.24800087e-01],
       [-2.92720423e-02,  6.63187253e-03,  7.91678485e-04, ...,
         8.78457059e-03,  1.75428931e-02,  1.71095244e-02],
       [-3.77106661e-02, -7.77956669e-02,  1.60040534e-02, ...,
         4.02972342e-04, -2.70474185e-02, -1.78457818e-02]])

In [35]:
columns32=['topic{}'.format(i) for i in range(svd.n_components)]
svd_topic_vectors=pd.DataFrame(svd_topic_vectors,index=index,columns=columns32)

In [36]:
svd_topic_vectors.head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic22,topic23,topic24,topic25,topic26,topic27,topic28,topic29,topic30,topic31
sms0,0.201169,0.002771,0.037234,0.010974,-0.019225,-0.052975,0.039145,-0.065587,0.011950,-0.082529,...,-0.024518,-0.016674,0.001497,0.002929,-0.013521,-0.047072,0.061672,-0.018237,-0.013267,0.007314
sms1,0.404380,-0.093875,-0.077513,0.050889,0.100057,0.047101,0.022874,0.064961,0.023051,-0.023995,...,0.006008,0.016093,-0.009995,0.021246,-0.006441,0.021068,0.014928,-0.033513,-0.035004,-0.011641
sms2!,-0.030459,-0.048104,0.090191,-0.067057,0.090746,-0.043146,-0.000284,-0.000999,-0.057004,0.051373,...,-0.029997,-0.004728,-0.020204,-0.026034,0.008701,0.035509,-0.031936,0.017356,-0.034313,0.007239
sms3,0.329048,-0.032768,-0.034568,-0.015813,0.052228,0.055726,-0.165577,-0.074179,0.062529,-0.107633,...,-0.000908,-0.009640,-0.038639,0.004286,-0.004476,-0.028598,-0.050049,-0.070882,-0.052402,-0.035095
sms4,0.002159,0.030857,0.038316,0.033874,-0.074668,-0.092527,-0.043713,0.061275,-0.044636,0.028772,...,-0.076065,0.021299,-0.054650,0.038878,-0.008421,-0.028016,-0.016697,-0.020853,0.044163,0.039013


In [37]:
x_train,x_test,y_train,y_test=train_test_split(svd_topic_vectors,sms.spam.values,test_size=0.2,random_state=271828)

In [38]:
lda=LDA(n_components=1)
lda=lda.fit(x_train,y_train)
sms['svd_spam']=lda.predict(svd_topic_vectors)
x_train.shape

(3869, 32)

In [39]:
round(float(lda.score(x_train,y_train)),3)

0.965

In [40]:
round(float(lda.score(x_test,y_test)),3)

0.973

In [41]:
from pugnlp.stats import Confusion
Confusion(sms['spam svd_spam'.split()])

svd_spam,0,1
spam,,
0,4164,35
1,125,513
